This document will compare the model report to the audit reports carried out by case-workers

Importing packages and read in CSVs:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
audit = pd.read_excel('s3://xxxxxxxxx/Audit.xlsx' )

mdl_SENSITIVE = pd.read_csv('s3://xxxxxxxxx/mdl_SENSITIVE.csv')

In [ ]:
print(len(audit), len(mdl_SENSITIVE))

Processing

In [ ]:
# Remove any entries that weren't put through the model

mdl_SENSITIVE = mdl_SENSITIVE[mdl_SENSITIVE['MODEL_PREDICTION'].notna()]

In [ ]:
len(mdl_SENSITIVE)

In [ ]:
# Take rows with the lowest model prediction e.g. -1 outliers and sort so those are at the top

mdl_SENSITIVE = mdl_SENSITIVE.sort_values(by= ['REF_NUM','MODEL_PREDICTION'], ascending=True)

In [ ]:
# drop duplicates in the audit file

audit = audit.loc[audit['REF_NUM'].notnull()].drop_duplicates('REF_NUM')

In [ ]:
audit['REF_NUM'] = audit['REF_NUM'].astype(int)

In [ ]:
for cols in mdl_SENSITIVE:
    print(cols)

In [ ]:
len(audit['REF_NUM']), len(mdl_SENSITIVE)

Let's merge to get a comparison

In [ ]:
mdl_audit_SENSITIVE = mdl_SENSITIVE.merge(audit, how='outer', on=['REF_NUM'], indicator=True)
mdl_audit_SENSITIVE = mdl_audit_SENSITIVE.dropna(axis=1, how='all')
len(mdl_audit_SENSITIVE)

In [ ]:
def agg_doc(df):
    merge_status = df['_merge'].unique()
    root_cause = df['Root Cause of Feedback'].unique()
    doc_score = df['SKLEARN_SCORE_ANOMALIES'].mean()
    '''If any line on doc ==-1 (outlier) then mark whole doc as outlier'''
    if (df['MODEL_PREDICTION']==-1).any():
#     if (len(df[df['MODEL_PREDICTION']==-1])>=2):

        return pd.DataFrame({'outlier_value': ['Outlier'], '_merge': merge_status, 'root_cause': root_cause,
                            'overall_doc_outlier_score': doc_score})
    
#     elif (len(df)==1) and ((df['MODEL_PREDICTION']==-1).any()):
        
#         return pd.DataFrame({'outlier_value': ['Outlier'], '_merge': merge_status, 'root_cause': root_cause})
    
    else:

        return pd.DataFrame({'outlier_value': ['Non_Outlier'], '_merge': merge_status, 'root_cause': root_cause})

In [ ]:
grouped_output = mdl_audit_SENSITIVE.groupby('REF_NUM').apply(agg_doc)

grouped_output = pd.DataFrame(grouped_output)

In [ ]:
len(grouped_output)

Adding a column to state whether overall doc has any lines that are detected as an outlier

In [ ]:
grouped_output.to_csv('s3://alpha-app-laa-fas-qc-dsa/mdl_audit_sensitive(lines grouped).csv')

In [ ]:
mdl_audit_SENSITIVE.to_csv('s3://alpha-app-laa-fas-qc-dsa/mdl_audit_sensitive.csv')